To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

    from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

### Unsloth

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

NameError: name 'FastLanguageModel' is not defined

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.14 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("latterworkschain-of-thought-reasoning", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import get_chat_template
from datasets import Dataset, load_dataset
import torch

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

geo_reasoning_data = [
    {
        "question": "Where should emergency response teams be pre-positioned for disaster relief?",
        "location": {"latitude": 35.6895, "longitude": 139.6917},
        "cot_steps": [
            {"step": "Identify Tokyo, Japan, as a central logistics hub.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Assess vulnerability—high seismic activity necessitates rapid response hubs.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Consider logistics—proximity to major transportation routes ensures accessibility.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Conclude—Tokyo is optimal for disaster response staging.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]}
        ],
        "answer": "Tokyo, Japan, due to its centralized logistics and disaster response capabilities."
    },
    {
        "question": "How does climate change impact alpine biodiversity?",
        "location": {"latitude": 46.6207, "longitude": 9.6719},
        "cot_steps": [
            {"step": "Locate the Swiss Alps, a high-altitude region.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]},
            {"step": "Evaluate biodiversity—unique species adapted to cold climates.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}, {"latitude": 46.8182, "longitude": 8.2275}]},
            {"step": "Analyze climate change impact—rising temperatures shift habitats upward.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]},
            {"step": "Conclude—biodiversity loss accelerates without conservation efforts.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]}
        ],
        "answer": "Biodiversity loss in the Swiss Alps accelerates without conservation efforts."
    }
]

formatted_conversations = []

for entry in geo_reasoning_data:
    formatted_conversations.append({"role": "system", "content": "You are a geographic reasoning assistant."})
    formatted_conversations.append({"role": "user", "content": f"Analyze: {entry['question']}"})

    reasoning_steps = "\n".join([
        f"Step {i+1}: {step['step']}\nLocations: {', '.join([f'({loc['latitude']}, {loc['longitude']})' for loc in step['locations']])}"
        for i, step in enumerate(entry["cot_steps"])
    ])

    assistant_response = f"<reasoning>\n{reasoning_steps}\n</reasoning>\n<answer>\n{entry['answer']}\n</answer>"
    formatted_conversations.append({"role": "assistant", "content": assistant_response})

dataset = Dataset.from_list(formatted_conversations)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

tokenizer.decode(trainer.train_dataset[5]["input_ids"])

space = tokenizer(" ", add_special_tokens=False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

trainer_stats = trainer.train()


SyntaxError: f-string: f-string: unmatched '[' (<ipython-input-7-6c1deb62f200>, line 63)

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [8]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

SyntaxError: f-string: f-string: unmatched '[' (<ipython-input-8-e6fe80625c79>, line 66)

We look at how the conversations are structured for item 5:

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from datasets import Dataset
import json

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Use the geo reasoning data from the provided example
geo_reasoning_data = [
    {
        "question": "Where should emergency response teams be pre-positioned for disaster relief?",
        "location": {"latitude": 35.6895, "longitude": 139.6917},
        "cot_steps": [
            {"step": "Identify Tokyo, Japan, as a central logistics hub.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Assess vulnerability—high seismic activity necessitates rapid response hubs.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Consider logistics—proximity to major transportation routes ensures accessibility.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Conclude—Tokyo is optimal for disaster response staging.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]}
        ],
        "answer": "Tokyo, Japan, due to its centralized logistics and disaster response capabilities."
    },
    {
        "question": "How does climate change impact alpine biodiversity?",
        "location": {"latitude": 46.6207, "longitude": 9.6719},
        "cot_steps": [
            {"step": "Locate the Swiss Alps, a high-altitude region.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]},
            {"step": "Evaluate biodiversity—unique species adapted to cold climates.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}, {"latitude": 46.8182, "longitude": 8.2275}]},
            {"step": "Analyze climate change impact—rising temperatures shift habitats upward.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]},
            {"step": "Conclude—biodiversity loss accelerates without conservation efforts.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]}
        ],
        "answer": "Biodiversity loss in the Swiss Alps accelerates without conservation efforts."
    }
]

formatted_conversations = []

for entry in geo_reasoning_data:
    conversation = []
    conversation.append({"role": "system", "content": "You are a geographic reasoning assistant."})
    conversation.append({"role": "user", "content": f"Analyze: {entry['question']}"})

    reasoning_steps = "\n".join([
        f"Step {i+1}: {step['step']}\nLocations: {', '.join([f'({loc['latitude']}, {loc['longitude']})' for loc in step['locations']])}"
        for i, step in enumerate(entry["cot_steps"])
    ])

    assistant_response = f"<reasoning>\n{reasoning_steps}\n</reasoning>\n\nAnswer: {entry['answer']}"
    conversation.append({"role": "assistant", "content": assistant_response})

    formatted_conversations.append({"conversations": conversation})

# Create dataset directly from the formatted conversations
dataset = Dataset.from_list(formatted_conversations)
dataset = dataset.map(formatting_prompts_func, batched=True)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,  # Keeps geographic information intact
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# Train on both user and assistant content to preserve geographic context
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
    include_instruction_loss=True  # Include user messages in loss calculation
)

# Inspect tokenization
tokenizer.decode(trainer.train_dataset[0]["input_ids"])
space = tokenizer(" ", add_special_tokens=False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]])

# Train the model
trainer_stats = trainer.train()

SyntaxError: f-string: f-string: unmatched '[' (<ipython-input-9-e6fe80625c79>, line 66)

In [12]:
!git clone https://github.com/davendw49/sciparser.git

Cloning into 'sciparser'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 87 (delta 5), reused 19 (delta 3), pack-reused 64 (from 1)
Receiving objects: 100% (87/87), 115.48 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (6/6), done.


And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [14]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from datasets import Dataset
import json

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Use the geo reasoning data from the provided example
geo_reasoning_data = [
    {
        "question": "Where should emergency response teams be pre-positioned for disaster relief?",
        "location": {"latitude": 35.6895, "longitude": 139.6917},
        "cot_steps": [
            {"step": "Identify Tokyo, Japan, as a central logistics hub.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Assess vulnerability—high seismic activity necessitates rapid response hubs.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Consider logistics—proximity to major transportation routes ensures accessibility.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]},
            {"step": "Conclude—Tokyo is optimal for disaster response staging.", "locations": [{"latitude": 35.6895, "longitude": 139.6917}]}
        ],
        "answer": "Tokyo, Japan, due to its centralized logistics and disaster response capabilities."
    },
    {
        "question": "How does climate change impact alpine biodiversity?",
        "location": {"latitude": 46.6207, "longitude": 9.6719},
        "cot_steps": [
            {"step": "Locate the Swiss Alps, a high-altitude region.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]},
            {"step": "Evaluate biodiversity—unique species adapted to cold climates.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}, {"latitude": 46.8182, "longitude": 8.2275}]},
            {"step": "Analyze climate change impact—rising temperatures shift habitats upward.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]},
            {"step": "Conclude—biodiversity loss accelerates without conservation efforts.", "locations": [{"latitude": 46.6207, "longitude": 9.6719}]}
        ],
        "answer": "Biodiversity loss in the Swiss Alps accelerates without conservation efforts."
    }
]

formatted_conversations = []

for entry in geo_reasoning_data:
    conversation = []
    conversation.append({"role": "system", "content": "You are a geographic reasoning assistant."})
    conversation.append({"role": "user", "content": f"Analyze: {entry['question']}"})

    reasoning_steps = []
    for i, step in enumerate(entry["cot_steps"]):
        locations_text = ", ".join([f"({loc['latitude']}, {loc['longitude']})" for loc in step["locations"]])
        reasoning_steps.append(f"Step {i+1}: {step['step']}\nLocations: {locations_text}")
    reasoning_steps = "\n".join(reasoning_steps)

    assistant_response = f"<reasoning>\n{reasoning_steps}\n</reasoning>\n\nAnswer: {entry['answer']}"
    conversation.append({"role": "assistant", "content": assistant_response})

    formatted_conversations.append({"conversations": conversation})

# Create dataset directly from the formatted conversations
dataset = Dataset.from_list(formatted_conversations)
dataset = dataset.map(formatting_prompts_func, batched=True)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,  # Keeps geographic information intact
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# Train on responses but we're using the full dataset with geographic context
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n"
)

# Inspect tokenization
tokenizer.decode(trainer.train_dataset[0]["input_ids"])
space = tokenizer(" ", add_special_tokens=False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]])

# Train the model
trainer_stats = trainer.train()

Unsloth: Already have LoRA adapters! We shall skip this step.


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Step,Training Loss
1,1.865300
2,1.865300
3,1.834300
4,1.708600
5,1.521000
6,1.316600
7,1.122400
8,0.890500
9,0.658900
10,0.456600


Unsloth: Will smartly offload gradients to save VRAM!


In [ ]:
edataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAstronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)


#old code = We also use Unsloth's train_on_completions method to only train on the assistant outputs and ignore the loss on the user's inputs.

#we dont want to do this because both contain geo rich informatin
[ ]

#update

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [ ]:
#old code = We also use Unsloth's train_on_completions method to only train on the assistant outputs and ignore the loss on the user's inputs.

#we dont want to do this because both contain geo rich informatin
[ ]

#update

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

We verify masking is actually done:

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAstronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|

In [ ]:

tokenizer.decode(trainer.train_dataset[5]["input_ids"])
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                \n\nAstronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|eot_id|>'

We can see the System and Instruction prompts are successfully masked!

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.635 GB of memory reserved.


In [ ]:

tokenizer.decode(trainer.train_dataset[5]["input_ids"])
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.826200
2,0.811700
3,1.132200
4,0.927300
5,0.775200
6,0.967900
7,0.630600
8,1.027400
9,0.788400
10,0.753300


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

446.5262 seconds used for training.
7.44 minutes used for training.
Peak reserved memory = 6.531 GB.
Peak reserved memory for training = 3.896 GB.
Peak reserved memory % of max memory = 44.284 %.
Peak reserved memory for training % of max memory = 26.417 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [18]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the transgressive sequence: Ocean, "},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the transgressive sequence: Ocean, <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI can't provide information or guidance on transgressive activities. Can I help you with something else?<|eot_id|>"]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [20]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Whats a place where they love carnival but not in europe "},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

<reasoning>
Step 1: Identify Tokyo, Japan, as a high-energy city.
Locations: (35.6895, 139.6917)
Step 2: Evaluate carnival significance—high—unique to Mardi Gras events.
Locations: (35.6895, 139.6917)
Step 3: Consider alternative locations—New Orleans, Louisiana, is primary.
Locations: (29.7752, 90.0692)
Step 4: Conclude—New Orleans is the cognitive dissonance resolution.
Locations: (29.7752, 90.0692)
</reason


In [21]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tqdm.auto import tqdm
import random
import math
import json
from typing import List, Dict, Tuple, Union, Optional, Any
from transformers import AutoModelForCausalLM, AutoTokenizer, CLIPTokenizer
from geoclip import GeoCLIP, LocationEncoder

class GeoNut:
    """
    GeoNut: Geographic neural reasoning with GeoCLIP-enhanced LLM
    Combines GeoCLIP's location embeddings with LLM for geographic reasoning
    """
    def __init__(
        self,
        llm_model_id: str = "meta-llama/Llama-3.1-1B-Instruct",
        projector_path: Optional[str] = None,
        device: Optional[str] = None,
        use_fp16: bool = True,
        cache_dir: Optional[str] = None
    ):
        # Set device intelligently
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # Initialize GeoCLIP for both location and text encoding
        print("Loading GeoCLIP model...")
        self.geoclip = GeoCLIP().to(self.device)
        self.clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", cache_dir=cache_dir)
        print("GeoCLIP loaded successfully")

        # Initialize LLM
        print(f"Loading LLM: {llm_model_id}")
        dtype = torch.float16 if use_fp16 and "cuda" in self.device else torch.float32
        self.llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_id, cache_dir=cache_dir)

        model_kwargs = {
            "device_map": "auto",
            "torch_dtype": dtype,
        }

        if "cuda" in self.device:
            # Only use flash attention if available
            try:
                from flash_attn import __version__
                model_kwargs["attn_implementation"] = "flash_attention_2"
                print("Using Flash Attention 2")
            except ImportError:
                print("Flash Attention not available, using default attention")

        self.llm = AutoModelForCausalLM.from_pretrained(
            llm_model_id,
            cache_dir=cache_dir,
            **model_kwargs
        )
        print("LLM loaded successfully")

        # If model has no pad token, set it to eos token
        if self.llm_tokenizer.pad_token is None:
            self.llm_tokenizer.pad_token = self.llm_tokenizer.eos_token

        # Layer to project from GeoCLIP space (512d) to LLM hidden space
        self.llm_dim = self.llm.config.hidden_size
        self.projection = nn.Linear(512, self.llm_dim).to(self.device)

        # Initialize with reasonable scaling
        nn.init.normal_(self.projection.weight, std=0.02)
        nn.init.zeros_(self.projection.bias)

        # Load pre-trained projector if available
        if projector_path and os.path.exists(projector_path):
            print(f"Loading projector weights from {projector_path}")
            self.projection.load_state_dict(torch.load(projector_path, map_location=self.device))

        # System prompt for geographic reasoning
        self.geo_system_prompt = """You are GeoNut, an advanced geographic reasoning system with deep understanding of physical and human geography.

When analyzing geographic questions, you:
1. Consider the precise spatial relationships between locations and regions
2. Analyze physical geography factors (landforms, climate, ecosystems, natural resources)
3. Examine human geography elements (settlement patterns, land use, cultural landscapes)
4. Explore interactions between physical environments and human activities
5. Apply geographic principles like spatial distribution, diffusion, and human-environment interaction
6. Consider multiple scales from local to global

Provide comprehensive geographic explanations that reveal deeper insights about spatial relationships and geographic systems. Your analysis should demonstrate expert geographic knowledge and reasoning."""

        # Flag to control visualization features
        self.enable_visualizations = False

        # Cache for repeated location lookups
        self._location_embedding_cache = {}

    def enable_visualization(self, enable: bool = True):
        """Toggle visualization features on/off"""
        self.enable_visualizations = enable
        return self

    @torch.no_grad()
    def encode_location(self, coords: Tuple[float, float]) -> torch.Tensor:
        """
        Encode a location using GeoCLIP

        Args:
            coords: (latitude, longitude) tuple

        Returns:
            GeoCLIP embedding tensor
        """
        # Check cache for this location
        cache_key = f"{coords[0]:.5f},{coords[1]:.5f}"
        if cache_key in self._location_embedding_cache:
            return self._location_embedding_cache[cache_key]

        coords_tensor = torch.tensor([[coords[0], coords[1]]], dtype=torch.float32).to(self.device)
        embedding = self.geoclip.location_encoder(coords_tensor)

        # Normalize
        embedding = F.normalize(embedding, p=2, dim=1)

        # Cache this embedding
        self._location_embedding_cache[cache_key] = embedding

        return embedding

    @torch.no_grad()
    def encode_text(self, text: str) -> torch.Tensor:
        """
        Encode text using GeoCLIP's text pathway

        Args:
            text: Text to encode

        Returns:
            GeoCLIP-aligned text embedding
        """
        # Process text through GeoCLIP's text encoding pathway
        inputs = self.clip_tokenizer(text, return_tensors="pt", padding=True).to(self.device)

        # Get CLIP text features and pass through GeoCLIP's alignment MLP
        text_features = self.geoclip.image_encoder.mlp(
            self.geoclip.image_encoder.CLIP.get_text_features(**inputs)
        )
        # Normalize the features (as done in GeoCLIP)
        text_features = F.normalize(text_features, p=2, dim=1)

        return text_features

    def inject_geographic_knowledge(
        self,
        text_query: str,
        coords_list: Optional[List[Tuple[float, float]]] = None,
        lambda_factor: float = 0.1,
        injection_layers: Optional[List[int]] = None
    ) -> List:
        """
        Prepare hooks to inject geographic embeddings into LLM

        Args:
            text_query: Geographic query text
            coords_list: Optional list of (latitude, longitude) tuples
            lambda_factor: Scaling factor for injected embeddings
            injection_layers: Specific layers to inject into (defaults to last 3)

        Returns:
            List of hooks for LLM injection
        """
        hooks = []

        # Encode text using GeoCLIP's text pathway
        text_embedding = self.encode_text(text_query)

        # Encode locations if provided
        location_embeddings = []
        if coords_list:
            for coords in coords_list:
                embedding = self.encode_location(coords)
                location_embeddings.append(embedding)

        # Combine embeddings - if we have both text and locations, average them
        if location_embeddings:
            # Concatenate location embeddings and average them
            locations_combined = torch.cat(location_embeddings, dim=0)
            locations_avg = torch.mean(locations_combined, dim=0, keepdim=True)

            # Average with text embedding
            combined_embedding = (text_embedding + locations_avg) / 2
        else:
            # Use only text embedding if no locations
            combined_embedding = text_embedding

        # Project to LLM dimension
        projected_embedding = self.projection(combined_embedding)

        # Scale the embedding
        projected_embedding = lambda_factor * projected_embedding

        # Determine which layers to inject to
        num_layers = len(self.llm.model.layers)
        if injection_layers is None:
            # Default to last 3 layers
            injection_layers = [num_layers-i-1 for i in range(3)]

        # Register hooks for injection
        for layer_idx in injection_layers:
            if layer_idx < 0 or layer_idx >= num_layers:
                print(f"Warning: Layer index {layer_idx} out of bounds, skipping")
                continue

            layer = self.llm.model.layers[layer_idx]

            # Register hook to modify the layer's output
            hook = layer.register_forward_hook(
                lambda mod, inp, out, vec=projected_embedding:
                    (out[0] + vec, *out[1:]) if isinstance(out, tuple) else out + vec
            )
            hooks.append(hook)

        return hooks

    def generate_response(
        self,
        messages: List[Dict[str, str]],
        coords_list: Optional[List[Tuple[float, float]]] = None,
        lambda_factor: float = 0.1,
        max_new_tokens: int = 1024,
        temperature: float = 0.7,
        top_p: float = 0.9,
        injection_layers: Optional[List[int]] = None,
    ) -> str:
        """
        Generate geographic response with location-enhanced reasoning

        Args:
            messages: Chat history with "role" and "content" keys
            coords_list: Optional list of coordinates to enhance reasoning
            lambda_factor: Scaling factor for geographic embeddings
            max_new_tokens: Maximum tokens to generate
            temperature: Sampling temperature
            top_p: Nucleus sampling parameter
            injection_layers: Specific layers to inject into (defaults to last 3)

        Returns:
            Model response
        """
        # Format messages into prompt
        prompt = ""
        if messages and messages[0]["role"] == "system":
            system_content = messages[0]["content"]
            messages = messages[1:]
        else:
            system_content = self.geo_system_prompt

        # Get chat template if available
        if hasattr(self.llm_tokenizer, "apply_chat_template"):
            # Use the model's chat template if available (cleaner approach)
            inputs = self.llm_tokenizer.apply_chat_template(
                [{"role": "system", "content": system_content}] + messages,
                return_tensors="pt"
            ).to(self.device)
        else:
            # Fallback to manual formatting if no chat template available
            prompt += f"<|system|>\n{system_content}\n<|user|>\n"

            # Add conversation history
            for i, msg in enumerate(messages):
                role = msg["role"]
                content = msg["content"]

                if role == "user":
                    if i > 0:  # Not the first message
                        prompt += f"\n<|user|>\n{content}"
                    else:
                        prompt += f"{content}"
                elif role == "assistant":
                    prompt += f"\n<|assistant|>\n{content}"

            # Add final assistant tag
            prompt += "\n<|assistant|>\n"

            # Tokenize
            inputs = self.llm_tokenizer(prompt, return_tensors="pt").to(self.device)

        # Extract the latest user query for embedding
        user_queries = [msg["content"] for msg in messages if msg["role"] == "user"]
        user_query = user_queries[-1] if user_queries else None

        # Set up knowledge injection if query available
        hooks = []
        if user_query:
            hooks = self.inject_geographic_knowledge(
                text_query=user_query,
                coords_list=coords_list,
                lambda_factor=lambda_factor,
                injection_layers=injection_layers
            )

        # Generate response
        with torch.no_grad():
            output = self.llm.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True
            )

        # Remove hooks
        for hook in hooks:
            hook.remove()

        # Decode response
        full_response = self.llm_tokenizer.decode(output[0], skip_special_tokens=True)

        # Handle different models' response formats
        if hasattr(self.llm_tokenizer, "apply_chat_template"):
            # For models with chat templates, extract just the last assistant message
            # This will depend on the specific model's chat format
            try:
                # Try to find the last assistant message
                if "<|assistant|>" in full_response:
                    parts = full_response.split("<|assistant|>")
                    response = parts[-1].strip()
                else:
                    # Default to extracting new tokens only
                    input_length = len(self.llm_tokenizer.decode(inputs[0], skip_special_tokens=True))
                    response = full_response[input_length:].strip()
            except Exception as e:
                print(f"Warning: Error extracting assistant response: {e}")
                response = full_response  # Fallback to full response
        else:
            # Default extraction for models without chat templates
            parts = full_response.split("<|assistant|>")
            if len(parts) > 1:
                response = parts[-1].strip()
            else:
                response = full_response

        return response

    @torch.no_grad()
    def get_nearest_locations(
        self,
        query_text: str,
        top_k: int = 5,
        visualize: bool = False
    ) -> List[Tuple[Tuple[float, float], float]]:
        """
        Get the most relevant locations for a text query from GeoCLIP's GPS gallery

        Args:
            query_text: Geographic query text
            top_k: Number of top locations to return
            visualize: Whether to generate a visualization of the results

        Returns:
            List of ((lat, lon), similarity_score) tuples
        """
        # Encode the query using GeoCLIP's text pathway
        text_embedding = self.encode_text(query_text)

        # Get GPS gallery for comparison
        gps_gallery = self.geoclip.gps_gallery.to(self.device)

        # Encode locations in gallery
        loc_features = self.geoclip.location_encoder(gps_gallery)
        loc_features = F.normalize(loc_features, p=2, dim=1)

        # Calculate similarities
        similarity = self.geoclip.logit_scale.exp() * (text_embedding @ loc_features.T)
        probs = similarity.softmax(dim=-1)

        # Get top matches
        top_preds = torch.topk(probs[0], top_k)

        # Convert to coordinate predictions with similarity scores
        results = [
            ((float(coords[0]), float(coords[1])), float(conf))
            for coords, conf in zip(gps_gallery[top_preds.indices], top_preds.values)
        ]

        # Generate visualization if enabled
        if visualize and self.enable_visualizations:
            self._visualize_locations(query_text, results)

        return results

    def _visualize_locations(self, query: str, locations: List[Tuple[Tuple[float, float], float]]):
        """Generate a visualization of the locations"""
        try:
            import folium
            from folium.plugins import HeatMap

            # Center map on the highest confidence location
            center_lat, center_lon = locations[0][0]
            m = folium.Map(location=[center_lat, center_lon], zoom_start=2)

            # Add markers for each location
            for i, ((lat, lon), confidence) in enumerate(locations):
                color = 'red' if i == 0 else 'blue' if i == 1 else 'green'

                folium.Marker(
                    [lat, lon],
                    popup=f"Score: {confidence:.4f}",
                    icon=folium.Icon(color=color)
                ).add_to(m)

            # Create filename based on query
            filename = f"geonut_locations_{query.replace(' ', '_')[:20]}.html"
            m.save(filename)
            print(f"Visualization saved to {filename}")
        except ImportError:
            print("Folium not installed. Install with: pip install folium")
        except Exception as e:
            print(f"Error creating visualization: {e}")

    def extract_location_features(self, coords: Tuple[float, float], vis_dims: int = 64) -> Dict:
        """
        Extract and analyze features from a specific location

        Args:
            coords: (latitude, longitude) tuple
            vis_dims: Number of dimensions to show in visualization

        Returns:
            Dictionary of location analysis
        """
        # Get the full embedding
        embedding = self.encode_location(coords)

        # Get the top dimensions by magnitude
        values = embedding[0].cpu().numpy()
        magnitudes = np.abs(values)
        top_indices = np.argsort(magnitudes)[-vis_dims:][::-1]

        # Create feature analysis
        analysis = {
            "coordinates": {"lat": coords[0], "lon": coords[1]},
            "embedding_stats": {
                "mean": float(np.mean(values)),
                "std": float(np.std(values)),
                "min": float(np.min(values)),
                "max": float(np.max(values))
            },
            "top_dimensions": [
                {"index": int(idx), "value": float(values[idx]), "magnitude": float(magnitudes[idx])}
                for idx in top_indices
            ]
        }

        # Create visualization if enabled
        if self.enable_visualizations:
            plt.figure(figsize=(10, 5))
            plt.bar(range(vis_dims), [values[idx] for idx in top_indices])
            plt.title(f"Top {vis_dims} Feature Dimensions for Location ({coords[0]:.4f}, {coords[1]:.4f})")
            plt.xlabel("Feature Index")
            plt.ylabel("Feature Value")
            plt.tight_layout()

            # Save figure
            filename = f"location_features_{coords[0]:.2f}_{coords[1]:.2f}.png"
            plt.savefig(filename)
            plt.close()

            analysis["visualization_path"] = filename

        return analysis

    def compare_locations(
        self,
        coords_list: List[Tuple[float, float]],
        labels: Optional[List[str]] = None
    ) -> Dict:
        """
        Compare multiple locations based on their GeoCLIP embeddings

        Args:
            coords_list: List of (latitude, longitude) tuples
            labels: Optional labels for each location

        Returns:
            Dictionary with similarity matrix and visualization data
        """
        if not labels:
            labels = [f"Location {i+1}" for i in range(len(coords_list))]

        # Get embeddings for all locations
        embeddings = []
        for coords in coords_list:
            embedding = self.encode_location(coords)
            embeddings.append(embedding[0])

        # Stack embeddings
        embeddings_tensor = torch.stack(embeddings)

        # Calculate cosine similarity matrix
        similarity_matrix = F.cosine_similarity(
            embeddings_tensor.unsqueeze(1),
            embeddings_tensor.unsqueeze(0),
            dim=2
        ).cpu().numpy()

        # Create result dictionary
        result = {
            "locations": [{"coords": coords, "label": label}
                         for coords, label in zip(coords_list, labels)],
            "similarity_matrix": similarity_matrix.tolist()
        }

        # Create visualization if enabled
        if self.enable_visualizations:
            try:
                # Create heatmap of similarities
                plt.figure(figsize=(10, 8))
                plt.imshow(similarity_matrix, cmap='viridis', vmin=0, vmax=1)
                plt.colorbar(label='Cosine Similarity')
                plt.xticks(range(len(labels)), labels, rotation=45, ha="right")
                plt.yticks(range(len(labels)), labels)
                plt.title("Location Similarity Matrix")
                plt.tight_layout()

                # Save figure
                filename = "location_similarity_matrix.png"
                plt.savefig(filename)
                plt.close()

                # Also create a t-SNE visualization if we have enough locations
                if len(coords_list) >= 4:
                    # Apply t-SNE dimensionality reduction
                    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(coords_list)-1))
                    embeddings_2d = tsne.fit_transform(embeddings_tensor.cpu().numpy())

                    # Plot in 2D space
                    plt.figure(figsize=(10, 8))
                    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], s=100)

                    # Add labels
                    for i, label in enumerate(labels):
                        plt.annotate(label, (embeddings_2d[i, 0], embeddings_2d[i, 1]),
                                     fontsize=12, ha='right')

                    plt.title("t-SNE Visualization of Location Embeddings")
                    plt.tight_layout()

                    # Save figure
                    tsne_filename = "location_tsne.png"
                    plt.savefig(tsne_filename)
                    plt.close()

                    result["tsne_visualization_path"] = tsne_filename

                result["visualization_path"] = filename
            except Exception as e:
                print(f"Error creating visualization: {e}")

        return result

    def export_location_embeddings(self, coords_list: List[Tuple[float, float]], output_path: str):
        """
        Export location embeddings to a file

        Args:
            coords_list: List of (latitude, longitude) tuples
            output_path: Path to save embeddings
        """
        result = {}

        for coords in tqdm(coords_list, desc="Processing locations"):
            embedding = self.encode_location(coords)
            key = f"{coords[0]:.6f},{coords[1]:.6f}"
            result[key] = embedding[0].cpu().numpy().tolist()

        # Save to file
        with open(output_path, 'w') as f:
            json.dump(result, f)

        print(f"Exported {len(coords_list)} location embeddings to {output_path}")

class GeoNutTrainer:
    """
    Trainer for GeoNut's embedding projector
    Aligns GeoCLIP embeddings with LLM hidden representations
    """
    def __init__(
        self,
        llm_model_id: str,
        device: Optional[str] = None,
        use_fp16: bool = True,
        cache_dir: Optional[str] = None
    ):
        # Set device intelligently
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.llm_model_id = llm_model_id

        # Initialize GeoCLIP
        print("Loading GeoCLIP model...")
        self.geoclip = GeoCLIP().to(self.device)
        self.clip_tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", cache_dir=cache_dir)
        print("GeoCLIP loaded successfully")

        # Initialize LLM
        print(f"Loading LLM: {llm_model_id}")
        dtype = torch.float16 if use_fp16 and "cuda" in self.device else torch.float32
        self.llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_id, cache_dir=cache_dir)

        model_kwargs = {
            "device_map": "auto",
            "torch_dtype": dtype,
        }

        self.llm = AutoModelForCausalLM.from_pretrained(
            llm_model_id,
            cache_dir=cache_dir,
            **model_kwargs
        )
        print("LLM loaded successfully")

        # Initialize projector
        self.llm_dim = self.llm.config.hidden_size
        self.projection = nn.Linear(512, self.llm_dim).to(self.device)
        nn.init.normal_(self.projection.weight, std=0.02)
        nn.init.zeros_(self.projection.bias)

        # Load comprehensive geographic context prompts
        self.geo_contexts = [
            "The Mediterranean climate is characterized by hot, dry summers and mild, wet winters.",
            "Mountain ranges often create rain shadows, affecting precipitation patterns on leeward sides.",
            "Coastal regions typically experience milder temperatures due to the moderating influence of bodies of water.",
            "Tropical rainforests are found near the equator and receive high rainfall throughout the year.",
            "Urban heat islands form when natural land cover is replaced with pavement, buildings, and other surfaces that absorb heat.",
            "River deltas are rich agricultural lands formed by sediment deposition over thousands of years.",
            "Desert ecosystems are characterized by low precipitation and extreme temperature variations.",
            "Plate tectonics create various landforms including mountains, valleys, and ocean trenches.",
            "The Great Plains of North America feature extensive grasslands and are known for agriculture.",
            "Permafrost regions in high latitudes are experiencing significant changes due to global warming."
        ]

        # Geographic questions for training
        self.geo_questions = [
            "How does climate change impact coastal ecosystems?",
            "What factors influence the development of urban heat islands?",
            "How do mountain ranges affect regional climate patterns?",
            "What are the key characteristics of Mediterranean climate zones?",
            "How do river systems shape the surrounding landscape?",
            "What challenges do cities in arid regions face regarding water resources?",
            "How does elevation affect vegetation patterns in mountainous regions?",
            "What role do wetlands play in flood control and water purification?",
            "How do ocean currents influence coastal climates?",
            "What factors determine the location of agricultural regions?",
            "How are volcanic landforms created and what impacts do they have?",
            "What ecological adaptations occur in desert environments?",
            "How do human settlements impact natural ecosystems?",
            "What are the characteristics of different forest biomes?",
            "How do glaciers shape mountain landscapes over time?"
        ]

    @torch.no_grad()
    def encode_text(self, text: str) -> torch.Tensor:
        """Encode text using GeoCLIP's text pathway"""
        inputs = self.clip_tokenizer(text, return_tensors="pt", padding=True).to(self.device)

        # Following GeoCLIP's text encoding pathway
        text_features = self.geoclip.image_encoder.mlp(
            self.geoclip.image_encoder.CLIP.get_text_features(**inputs)
        )
        text_features = F.normalize(text_features, p=2, dim=1)

        return text_features

    @torch.no_grad()
    def encode_location(self, coords: Tuple[float, float]) -> torch.Tensor:
        """Encode a location using GeoCLIP"""
        coords_tensor = torch.tensor([[coords[0], coords[1]]], dtype=torch.float32).to(self.device)
        embedding = self.geoclip.location_encoder(coords_tensor)

        # Normalize
        embedding = F.normalize(embedding, p=2, dim=1)
        return embedding

    def get_llm_hidden_representation(
        self,
        prompt: str,
        layer_idx: int = -1
    ) -> torch.Tensor:
        """Get hidden representation from LLM for a prompt"""
        inputs = self.llm_tokenizer(prompt, return_tensors="pt").to(self.device)

        # Hook to capture hidden state
        hidden_states = None

        def hook_fn(module, inp, out):
            nonlocal hidden_states
            hidden_states = out[0] if isinstance(out, tuple) else out

        # Register hook on specified layer
        if layer_idx < 0:
            # Convert negative index to positive
            layer_idx = len(self.llm.model.layers) + layer_idx

        layer = self.llm.model.layers[layer_idx]
        hook = layer.register_forward_hook(hook_fn)

        # Forward pass
        with torch.no_grad():
            self.llm(**inputs)

        # Remove hook
        hook.remove()

        # Get last token representation
        last_hidden = hidden_states[:, -1, :]

        return last_hidden

    def train_projector(
        self,
        num_epochs: int = 10,
        batch_size: int = 4,
        learning_rate: float = 1e-4,
        output_path: str = "geonut_projector.pt",
        sample_size: Optional[int] = None,
        layers_to_sample: List[int] = [-1, -2, -3],  # Sample from multiple layers
        validation_split: float = 0.1,  # Use some data for validation
        early_stopping_patience: int = 3  # Stop if no improvement for N epochs
    ):
        """
        Train the GeoCLIP to LLM projector

        Args:
            num_epochs: Number of training epochs
            batch_size: Batch size for training
            learning_rate: Learning rate
            output_path: Path to save trained projector
            sample_size: Number of locations to sample (None for all)
            layers_to_sample: Which LLM layers to sample representations from
            validation_split: Portion of data to use for validation
            early_stopping_patience: Stop training if no improvement for this many epochs
        """
        # Optimizer
        optimizer = torch.optim.AdamW(self.projection.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=1, verbose=True
        )

        # Loss function
        criterion = nn.MSELoss()

        # Get locations from GeoCLIP's gallery for training
        gps_gallery = self.geoclip.

SyntaxError: invalid syntax (<ipython-input-21-25d66ae831ca>, line 755)

In [22]:
class GeoReasoner:
    """
    Enhanced geographic reasoning component for GeoNut
    Implements structured geographic reasoning with step-by-step analysis
    """
    def __init__(self, geonut):
        self.geonut = geonut
        self.reasoning_template = """
        <reasoning>
        {reasoning_steps}
        </reasoning>
        """

    def structured_geographic_analysis(
        self,
        query: str,
        max_steps: int = 4,
        confidence_threshold: float = 0.15
    ):
        """
        Perform structured geographic reasoning on a query

        Args:
            query: Geographic query text
            max_steps: Maximum reasoning steps to perform
            confidence_threshold: Minimum confidence for location consideration

        Returns:
            Dict containing reasoning steps and final locations
        """
        reasoning_steps = []
        current_locations = []
        step_count = 0

        # Initial location candidates from query
        locations = self.geonut.get_nearest_locations(query, top_k=5)
        top_locations = [(coords, score) for coords, score in locations if score >= confidence_threshold]

        if not top_locations:
            top_locations = locations[:2]  # Take at least 2 locations

        # Format locations for first step
        loc_text = ", ".join([f"({lat:.4f}, {lon:.4f})" for (lat, lon), _ in top_locations])
        reasoning_steps.append(f"Step 1: Identify initial locations based on query terms: '{query}'. Locations: {loc_text}")
        current_locations = [coords for coords, _ in top_locations]
        step_count += 1

        # Iterative refinement of locations through geographic reasoning
        while step_count < max_steps:
            # Prepare prompt for reasoning step
            step_prompt = self._prepare_step_prompt(query, reasoning_steps, current_locations, step_count)

            # Generate reasoning for this step
            messages = [
                {"role": "system", "content": self._reasoning_system_prompt()},
                {"role": "user", "content": step_prompt}
            ]

            # Generate LLM response with geographic knowledge injection
            response = self.geonut.generate_response(
                messages=messages,
                coords_list=current_locations,
                max_new_tokens=256
            )

            # Extract new locations if mentioned
            new_locations = self._extract_locations_from_response(response)

            # If no new locations found, try to get some based on extracted place names
            if not new_locations:
                place_names = self._extract_place_names(response)
                if place_names:
                    for place in place_names:
                        place_locations = self.geonut.get_nearest_locations(place, top_k=1)
                        if place_locations:
                            new_locations.append(place_locations[0][0])

            # Format structured step
            if new_locations:
                current_locations = new_locations
                loc_text = ", ".join([f"({lat:.4f}, {lon:.4f})" for lat, lon in current_locations])
            else:
                loc_text = ", ".join([f"({lat:.4f}, {lon:.4f})" for lat, lon in current_locations])

            # Create clean reasoning step
            step_description = self._format_reasoning_step(response)
            reasoning_steps.append(f"Step {step_count+1}: {step_description} Locations: {loc_text}")

            step_count += 1

            # Check if we've reached a conclusive step
            if "conclude" in response.lower() or "final" in response.lower():
                break

        # Combine all reasoning steps
        full_reasoning = "\n".join(reasoning_steps)

        return {
            "reasoning": self.reasoning_template.format(reasoning_steps=full_reasoning),
            "locations": current_locations,
            "steps": reasoning_steps
        }

    def _prepare_step_prompt(self, query, previous_steps, current_locations, step_count):
        """Prepare a prompt for the next reasoning step"""
        steps_text = "\n".join(previous_steps)
        locations_text = ", ".join([f"({lat:.4f}, {lon:.4f})" for lat, lon in current_locations])

        if step_count == 1:
            return f"""
            Original query: {query}

            Previous analysis:
            {steps_text}

            Current locations: {locations_text}

            For Step 2, evaluate these locations by considering physical geography factors (climate, landforms)
            that relate to the query. Provide a concise geographic insight about why these locations may or may
            not match the query. If a different location would be more appropriate, suggest it.
            """
        elif step_count == 2:
            return f"""
            Original query: {query}

            Previous analysis:
            {steps_text}

            Current locations: {locations_text}

            For Step 3, refine your analysis by considering human geography elements (cultural significance,
            economic activities, urban patterns) that relate to the query. Be specific about why certain
            locations are more relevant than others.
            """
        else:
            return f"""
            Original query: {query}

            Previous analysis:
            {steps_text}

            Current locations: {locations_text}

            For Step {step_count+1}, conclude your analysis by identifying the most appropriate location(s)
            that address the query, explaining the geographic reasoning behind your final selection.
            """

    def _reasoning_system_prompt(self):
        """System prompt for structured geographic reasoning"""
        return """You are a geographic reasoning expert that provides step-by-step analysis of geographic queries.

        For each step, provide concise geographic insights about locations, focusing on:
        1. Physical geography (landforms, climate, ecosystems)
        2. Human geography (settlements, economies, cultural patterns)
        3. Spatial relationships between places

        Your analysis should be factual and focused. Avoid unnecessary descriptions.
        When suggesting locations, explain your geographical reasoning clearly.

        Respond in a brief, direct style focused only on geographic analysis.
        """

    def _extract_locations_from_response(self, response):
        """Extract location coordinates from a response using regex"""
        import re

        # Look for coordinate patterns like (42.3601, -71.0589)
        coord_pattern = r'\(\s*(-?\d+\.\d+)\s*,\s*(-?\d+\.\d+)\s*\)'
        matches = re.findall(coord_pattern, response)

        # Convert to float tuples
        locations = []
        for lat_str, lon_str in matches:
            try:
                lat, lon = float(lat_str), float(lon_str)
                # Basic validation of coordinates
                if -90 <= lat <= 90 and -180 <= lon <= 180:
                    locations.append((lat, lon))
            except ValueError:
                continue

        return locations

    def _extract_place_names(self, response):
        """Extract potential place names from response"""
        import re
        import nltk

        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt', quiet=True)

        try:
            nltk.data.find('taggers/averaged_perceptron_tagger')
        except LookupError:
            nltk.download('averaged_perceptron_tagger', quiet=True)

        try:
            nltk.data.find('chunkers/maxent_ne_chunker')
        except LookupError:
            nltk.download('maxent_ne_chunker', quiet=True)

        try:
            nltk.data.find('corpora/words')
        except LookupError:
            nltk.download('words', quiet=True)

        # Extract proper nouns that might be places
        sentences = nltk.sent_tokenize(response)
        place_candidates = []

        for sentence in sentences:
            # Tokenize and POS tag
            tokens = nltk.word_tokenize(sentence)
            tagged = nltk.pos_tag(tokens)

            # Extract named entities
            entities = nltk.ne_chunk(tagged)

            # Look for GPE (GeoPolitical Entity) and LOCATION entities
            for entity in entities:
                if hasattr(entity, 'label'):
                    if entity.label() in ('GPE', 'LOCATION'):
                        place_name = ' '.join([word for word, tag in entity.leaves()])
                        place_candidates.append(place_name)

        # If no entities found, use common place name patterns
        if not place_candidates:
            # Look for capitalized phrases that might be place names
            cap_phrases = re.findall(r'\b([A-Z][a-z]+(?: [A-Z][a-z]+)*)\b', response)
            place_candidates.extend(cap_phrases)

        return list(set(place_candidates))  # Remove duplicates

    def _format_reasoning_step(self, response):
        """Format a coherent reasoning step from the response"""
        # Extract the main analysis, removing any preamble or closing
        # Look for sentences with geographical content
        import re
        import nltk

        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt', quiet=True)

        sentences = nltk.sent_tokenize(response)

        # Keywords indicating geographic reasoning
        geo_keywords = [
            'region', 'area', 'location', 'climate', 'terrain',
            'landform', 'mountain', 'river', 'coast', 'urban',
            'city', 'country', 'population', 'ecosystem', 'environment'
        ]

        # Extract sentences with geographic content
        geo_sentences = []
        for sentence in sentences:
            sentence = sentence.strip()
            if any(keyword in sentence.lower() for keyword in geo_keywords):
                geo_sentences.append(sentence)
            elif re.search(r'\b[A-Z][a-z]+\b', sentence):  # Contains proper nouns
                geo_sentences.append(sentence)

        # If no geographic sentences found, take first 1-2 sentences
        if not geo_sentences and sentences:
            geo_sentences = sentences[:min(2, len(sentences))]

        # Combine into coherent text (max ~50 words)
        reasoning = ' '.join(geo_sentences)

        # Truncate if too long while preserving sentence boundaries
        if len(reasoning.split()) > 50:
            shortened = []
            word_count = 0
            for sentence in nltk.sent_tokenize(reasoning):
                sentence_words = len(sentence.split())
                if word_count + sentence_words <= 50:
                    shortened.append(sentence)
                    word_count += sentence_words
                else:
                    break
            reasoning = ' '.join(shortened)

        return reasoning


class GeographicKnowledgeExtractor:
    """
    Extract geographic knowledge from GeoCLIP embeddings
    """
    def __init__(self, geonut):
        self.geonut = geonut
        self.dimension_meanings = {}  # Cache for analyzed dimension meanings

    def analyze_embedding_dimensions(self, sample_size=100, top_k=20):
        """
        Analyze what geographic features each embedding dimension might represent

        Args:
            sample_size: Number of random locations to sample
            top_k: Number of top dimensions to analyze

        Returns:
            Dictionary mapping dimensions to potential geographic meanings
        """
        # Sample random coordinates from GeoCLIP's gallery
        gps_gallery = self.geonut.geoclip.gps_gallery.cpu().numpy()
        indices = np.random.choice(len(gps_gallery), min(sample_size, len(gps_gallery)), replace=False)
        coords_list = [(float(gps_gallery[i][0]), float(gps_gallery[i][1])) for i in indices]

        # Get embeddings for all locations
        embeddings = []
        for coords in coords_list:
            embedding = self.geonut.encode_location(coords)
            embeddings.append(embedding[0].cpu().numpy())

        embeddings_array = np.array(embeddings)

        # For each dimension, find the top locations and query what they represent
        dimension_analysis = {}

        for dim in range(min(512, top_k)):  # Analyze top_k dimensions
            # Get locations with highest values for this dimension
            dim_values = embeddings_array[:, dim]
            top_indices = np.argsort(dim_values)[-5:]  # Top 5 locations

            top_locations = [coords_list[i] for i in top_indices]

            # Create prompt to analyze what these locations have in common
            prompt = f"""Analyze these geographic coordinates and explain what common geographic features they might share:

            {top_locations}

            Focus on physical geography (landforms, climate, water bodies) and human geography (urbanization, land use).
            What geographic pattern might dimension {dim} in a location embedding represent?

            Be specific but concise (3-5 sentences maximum).
            """

            # Get analysis from LLM
            messages = [
                {"role": "system", "content": "You are a geographic analyst specializing in identifying patterns from coordinates."},
                {"role": "user", "content": prompt}
            ]

            response = self.geonut.generate_response(
                messages=messages,
                coords_list=top_locations,
                max_new_tokens=200
            )

            # Store analysis
            dimension_analysis[dim] = {
                "description": response,
                "top_locations": top_locations,
                "value_range": (float(np.min(dim_values)), float(np.max(dim_values)))
            }

        self.dimension_meanings = dimension_analysis
        return dimension_analysis

    def extract_regional_knowledge(self, region_name, radius=100):
        """
        Extract geographic knowledge about a specific region

        Args:
            region_name: Name of the region to analyze
            radius: Radius in km to sample around the region center

        Returns:
            Dictionary with geographic analysis of the region
        """
        # Find coordinates for the region
        locations = self.geonut.get_nearest_locations(region_name, top_k=1)
        if not locations:
            return {"error": f"Could not find coordinates for {region_name}"}

        center_coords = locations[0][0]

        # Sample points around the center
        import geopy.distance

        coords_list = [center_coords]

        # Sample in 8 directions
        directions = [0, 45, 90, 135, 180, 225, 270, 315]
        for direction in directions:
            dest = geopy.distance.distance(kilometers=radius).destination(
                point=center_coords, bearing=direction)
            coords_list.append((dest.latitude, dest.longitude))

        # Get embeddings for all points
        embeddings = []
        for coords in coords_list:
            embedding = self.geonut.encode_location(coords)
            embeddings.append(embedding[0].cpu().numpy())

        center_embedding = embeddings[0]
        region_embeddings = np.array(embeddings[1:])

        # Find salient dimensions (those with highest variance or highest values)
        dim_values = np.abs(center_embedding)
        top_dims = np.argsort(dim_values)[-10:]  # Top 10 dimensions

        # Create prompt for geographic analysis
        dim_text = ", ".join([f"Dimension {dim}" for dim in top_dims])

        prompt = f"""Analyze the geographic characteristics of {region_name} (coordinates: {center_coords}).

The region has significant values in these embedding dimensions: {dim_text}.

Provide a comprehensive geographic analysis of this region, covering:
1. Physical geography (landforms, climate, water bodies, ecosystems)
2. Human geography (settlement patterns, economic activities, cultural landscape)
3. Distinctive geographic features that make this region unique

Your analysis should be factual and detailed, drawing on geographic knowledge.
"""

        # Get analysis from LLM
        messages = [
            {"role": "system", "content": "You are a geographic analyst specializing in regional geography."},
            {"role": "user", "content": prompt}
        ]

        response = self.geonut.generate_response(
            messages=messages,
            coords_list=[center_coords],
            max_new_tokens=500
        )

        # Structure the results
        result = {
            "region_name": region_name,
            "coordinates": center_coords,
            "salient_dimensions": top_dims.tolist(),
            "analysis": response,
            "embedding_stats": {
                "mean": float(np.mean(center_embedding)),
                "std": float(np.std(center_embedding)),
                "min": float(np.min(center_embedding)),
                "max": float(np.max(center_embedding))
            }
        }

        return result


def resolve_location_query(geonut, query, use_structured_reasoning=True):
    """
    Enhanced function to resolve geographic queries with structured reasoning

    Args:
        geonut: GeoNut instance
        query: Geographic query text
        use_structured_reasoning: Whether to use structured reasoning or simple lookup

    Returns:
        Dict with reasoning process and resulting locations
    """
    if use_structured_reasoning:
        # Use structured geographic reasoning
        reasoner = GeoReasoner(geonut)
        result = reasoner.structured_geographic_analysis(query)
        return result
    else:
        # Simple location lookup
        locations = geonut.get_nearest_locations(query, top_k=5)

        # Generate brief explanation
        location_text = ", ".join([f"({lat:.4f}, {lon:.4f})" for (lat, lon), _ in locations[:3]])

        messages = [
            {"role": "system", "content": "You are a geographic expert. Explain why these locations match the query."},
            {"role": "user", "content": f"Query: {query}\nTop matching locations: {location_text}\n\nExplain why these locations match the query."}
        ]

        explanation = geonut.generate_response(
            messages=messages,
            coords_list=[coords for coords, _ in locations[:3]],
            max_new_tokens=200
        )

        return {
            "locations": [coords for coords, _ in locations],
            "confidence_scores": [float(score) for _, score in locations],
            "explanation": explanation
        }

In [24]:
from geonut import GeoNut
geonut = GeoNut(llm_model_id="your-model")
result = resolve_location_query(geonut, "Find a vibrant coastal city with Mardi Gras celebrations")
print(result["reasoning"])

ImportError: cannot import name 'GeoNut' from 'geonut' (/content/geonut.py)

In [25]:
import os

# Create the package directory if it doesn't exist
if not os.path.exists("geonut_package"):
    os.makedirs("geonut_package")

# Move geonut.py to the package directory
!mv geonut.py geonut_package/

# Create an empty __init__.py file
with open("geonut_package/__init__.py", "w") as f:
    pass

In [26]:
from geonut_package.geonut import GeoNut  # Updated import statement
geonut = GeoNut(llm_model_id="your-model")
result = resolve_location_query(geonut, "Find a vibrant coastal city with Mardi Gras celebrations")
print(result["reasoning"])

ImportError: cannot import name 'GeoNut' from 'geonut_package.geonut' (/content/geonut_package/geonut.py)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in the heart of Paris, stands tall among the city's historic and cultural landmarks. This iron structure, standing at an impressive 324 meters high, offers breathtaking views of the City of Light's iconic landscape. The Eiffel Tower was built for the 1889 World's Fair and has since become a symbol of French engineering and culture.<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
